# Load the metada with titles and count the number of words 

In [ ]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib as mpl
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
from scipy.stats import bootstrap
import statsmodels.api as sm
import statsmodels.formula.api as smf

params = {
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "text.usetex": False,
}

mpl.rcParams.update(params)


import warnings

warnings.filterwarnings("ignore")


In [ ]:
video_metadatas = pd.read_feather(
    "./data/yt_metadata_helper.feather",
    columns=[
        "categories",
        "upload_date",
        "duration",
        "like_count",
        "dislike_count",
        "view_count",
        "channel_id",
    ],
)

In [ ]:
import re

# we need to improve the hw1 method a bit because we want to make sure that titles with a lot of spaces
# don't have too many words and if there is a type (e.g "Welcome,in my video" is counted as 4 words and not 3)
def count_words_simple(x: str):
    new_x = x.replace(",", " ")
    new_x = re.sub(" +", " ", new_x)
    return len(new_x.split(" "))


In [ ]:
print(count_words_simple("Salut     je suis la"))
print(count_words_simple("Salut,je suis la"))


In [ ]:
titles_metadata = pd.read_feather(
    "./data/yt_metadata_title_helper.feather", columns=["title"]
)


In [ ]:
titles_metadata.head()


In [ ]:
video_metadatas.head()


In [ ]:
# titles_metadata.drop('categories', axis=1, inplace=True)
# titles_metadata


In [ ]:
titles_nb_words = titles_metadata
titles_nb_words.title = titles_metadata["title"].apply(lambda x: count_words_simple(x))


In [ ]:
titles_nb_words.rename(columns={"title": "nb_words"}, inplace=True)


In [ ]:
video_metadatas_titles = pd.concat(
    [titles_nb_words, video_metadatas], axis=1, ignore_index=False
)


In [ ]:
video_metadatas_titles.sample(7)


In [ ]:
videos_per_nb_words = video_metadatas_titles.groupby("nb_words")["view_count"].size()
videos_per_nb_words


we delete the group with less than a thresholds since it biases the graph (After 23 words in the title there are not enough videos to have consistent results)

In [ ]:
view_mean_per_nb_words = video_metadatas_titles.groupby("nb_words")[
    "view_count"
].mean()[:23]
view_mean_per_nb_words


In [ ]:
view_mean_per_nb_words.index.values


# We have to be careful of YTbers with many views, if they have long titles it biases the result 

In [ ]:
x_ticks = view_mean_per_nb_words.index.values[::2]
ax = view_mean_per_nb_words.plot.bar(rot=0)
ax.set_xticks(x_ticks)
ax.legend()


In [ ]:
words_per_cat = video_metadatas_titles.groupby("categories").agg({"nb_words": "mean"})

words_per_cat.nb_words


In [ ]:
words_per_cat.index


In [ ]:
print("The x axis is :\n", words_per_cat.index.values[1:])
plt.xticks(fontsize=6)
plt.bar([index[:5] for index in words_per_cat.index[1:]], words_per_cat.nb_words[1:])
